## Data

### Done

1. load using pandas

## State Representation

### Done

1. Transform categorical idices
2. normalise cost

### TODO

1. continuous features bucketised ?
2. weights (real values floating point numbers, normalised (?))



In [125]:
import pandas as pd
pd.options.display.max_columns = None
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [126]:
raw_data = pd.read_csv('./src/data/insurance.csv')
raw_features = pd.DataFrame(raw_data, columns=raw_data.columns.drop('charges'))
raw_labels = pd.DataFrame(raw_data.charges)

In [127]:
# we use this to map the categorical features into indices, we can map back later for presentation
def extract_category_mappings(dataframe, category_columns):
    to_map = lambda uniques: {v:k[0] for k, v in np.ndenumerate(uniques)}
    return [(to_map(dataframe[col].unique()), col) for col in category_columns]

In [128]:
mappings = extract_category_mappings(raw_features, ['sex', 'smoker', 'region'])

In [129]:
# transform the categorical features to binary representations of their indices;
# we might want to bucketize the continuous features?
def transform_features(dataframe, mappings):
    transformed_df = pd.DataFrame(dataframe, columns=['age', 'bmi', 'children'])
    
    for mapping, name in mappings:
        for i in mapping.values():
            transformed_df[name + str(i)] = dataframe[name].apply(lambda x: int(mapping[x] == i))

    return transformed_df

In [130]:
features = transform_features(raw_features, mappings)

In [131]:
scaler = preprocessing.MinMaxScaler()

def normalize(series, columns, scaler):
    return scaler.fit_transform(series[columns])

def denormalize(series, columns, scaler):
    return scaler.inverse_transform(series[columns])

labels = raw_labels
labels['charges'] = normalize(raw_labels, ['charges'], scaler)
print(labels.head())


    charges
0  0.251611
1  0.009636
2  0.053115
3  0.333010
4  0.043816


In [132]:
X_train, X_test, y_train, y_test = train_test_split(features, labels)

In [133]:
print(X_train.head())
print(X_test.head())
print(y_train.head())
print(y_test.head())

     age     bmi  children  sex0  sex1  smoker0  smoker1  region0  region1  \
305   29  33.345         2     0     1        0        1        0        0   
278   59  31.790         2     0     1        0        1        0        1   
579   25  23.465         0     1     0        0        1        0        0   
348   36  29.920         1     1     0        0        1        0        1   
679   49  23.180         2     1     0        0        1        0        0   

     region2  region3  
305        1        0  
278        0        0  
579        0        1  
348        0        0  
679        1        0  
      age     bmi  children  sex0  sex1  smoker0  smoker1  region0  region1  \
1312   34  42.900         1     0     1        0        1        1        0   
739    29  35.500         2     0     1        1        0        1        0   
766    47  32.300         1     0     1        0        1        1        0   
1292   21  23.210         0     0     1        0        1        0     

In [134]:
#TODO warning, actually setting values on y_train if we run this
#test_denormalized_labels = y_train
#test_denormalized_labels['charges'] = denormalize(y_train, ['charges'], scaler)

#print(test_denormalized_labels.head())



# we probably only actually care about denormalising on predictions

,charges
305,0.292433
278,0.188463
579,0.033275
348,0.069533
679,0.144216


In [136]:
def predict(features, weights):
    return np.dot(features, weights)